In [57]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'  # template url for the search query
    position = position.replace(' ', '+')  # replace spaces in position with plus signs for url
    location = location.replace(' ', '+')  # replace spaces in location with plus signs for url
    url = template.format(position, location)  # insert position and location into url template
    return url


def get_record(card):
    """Extract job data from a single record"""
    
    job_title = card.h2.a.get('title')  # job title
    company = card.find('span', 'company').text.strip()  # company name
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')  # job location
    post_date = card.find('span', 'date').text  # posting date
    today = datetime.today().strftime('%Y-%m-%d')  # current date in yyyy-mm-dd format
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')  # job summary
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')  # url of job posting

    # check if job salary information exists and store it
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''
        
    # create a tuple of the job information
    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []  # list to store job information
    url = get_url(position, location)  # generate url for job search
    
    # extract job information from multiple pages of search results
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')  # find all job posting cards on page
        for card in cards:
            record = get_record(card)  # extract job information from card
            records.append(record)  # add job information to list
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
            # get url for next page of search results
        except AttributeError:
            break
        
    # save the job information to a CSV file
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        # write header row to CSV file
        writer.writerows(records)
        # write job information to CSV file


In [58]:
main('senior developer', 'dallas tx')